In [10]:
from kafka import KafkaProducer
from pyspark.sql import SparkSession 
from pyspark.sql.functions import *
import time, csv
from json import dumps

In [11]:
spark = SparkSession.builder \
        .config('spark.jars.packages', 'org.apache.spark:spark-streaming-kafka-0-10_2.12:3.1.2,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2')\
        .getOrCreate() 
df_uber = spark.read.csv("./data/uber.csv")


22/02/01 12:27:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [14]:
# convert to json with format key-value push to kafka
# using "struct" funtion to nested many columns into one column
# df = df_uber.select(to_json(struct("*")).alias("value"))
# df.show(5, False)

df = df_uber.select(concat_ws(",",col("_c0"), col("_c1"), col("_c2"), col("_c3")).alias("value"))
df.show(5, False)

+-------------------------------------------+
|value                                      |
+-------------------------------------------+
|2014-08-01 00:00:00,40.729,-73.9422,B02598 |
|2014-08-01 00:00:00,40.7476,-73.9871,B02598|
|2014-08-01 00:00:00,40.7424,-74.0044,B02598|
|2014-08-01 00:00:00,40.751,-73.9869,B02598 |
|2014-08-01 00:00:00,40.7406,-73.9902,B02598|
+-------------------------------------------+
only showing top 5 rows



In [15]:
# df.selectExpr("CAST(value AS STRING)") 
#     .write \
#     .format("kafka") \
#     .option("kafka.bootstrap.servers", "localhost:9092")\
#     .option("topic", "TestTopic")\
#     .save()

df.write \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092")\
    .option("topic", "TestTopic")\
    .save()

spark.stop()

In [5]:
# TOPCIC_NAME = "TestUber"
# BOOTSTRAP_SERVER = "localhost:9092"

# producer = KafkaProducer(bootstrap_servers=BOOTSTRAP_SERVER,value_serializer=lambda x:dumps(x).encode('utf-8'))

# message = None
# count_message=0
# with open("./data/uber.csv") as file:
#     reader = csv.reader(file, delimiter=",")
#     for messages in reader:
#         message = ",".join(messages)
#         count_message+=1
#         print("Message", count_message , ":", message)
#         producer.send(TOPCIC_NAME, message)
#         time.sleep(1)

In [ ]:
#Kafak Consumer

# kafka_consumer_obj = KafkaConsumer(KAFKA_OUTPUT_TOPIC_NAME,bootstrap_servers=KAFKA_SERVER,auto_offset_reset='latest',enable_auto_comit=True)
# print("Reading messages from Kfaka Topic")
# message_list = []
# counter = 0
# for message in kafka_consumer_obj:
#  print("Key: ",message.key)
#  output_message = message.value
#  print(type(message.value))
#  print("Message received: ",output_message)
#  message_list.append(output_message)
#  time.sleep(2)